In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
import sys
import transformers
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
# print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))
print(sys.version)

2.4.0
True
1
NVIDIA A100-PCIE-40GB
3.10.15 (main, Oct  3 2024, 07:27:34) [GCC 11.2.0]


In [1]:
from langchain_community.document_loaders import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma, FAISS
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, load_from_disk
import time
import json
import transformers

device="cuda"

# 加载文档
def load_documents(directory='/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/wiki.train.txt'):
    if "popQA" in directory:
        text_splitter=RecursiveCharacterTextSplitter(separators=["\n"])
        with open(directory, 'r', encoding='utf-8') as file:
            texts=file.readlines()
        decuments=text_splitter.create_documents(texts=texts)   
        return decuments
    loader = TextLoader(directory)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
    text_splitter=RecursiveCharacterTextSplitter(separators="\n")
    split_docs = text_splitter.split_documents(documents)
    return split_docs


# 加载embedding模型
def load_embedding_model(model_path='/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/models/bge-large-en-v1.5'):
    embedding_model = HuggingFaceEmbeddings(  # embedding 模型
        model_name=model_path,
        model_kwargs={'device': device},
        encode_kwargs={'normalize_embeddings': True}
    )
    return embedding_model


def load_llama(model_path='/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/models/Llama-2-7b-chat-hf'):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    tokenizer.pad_token = tokenizer.eos_token
    base_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        return_dict=True,
        torch_dtype=torch.float16,
        device_map={"": device},
        low_cpu_mem_usage=True
    )
    model = base_model.eval()
    return model, tokenizer
def load_Qwen(model_path="/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/models/Qwen2.5-7B-Instruct"):
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype="auto",
        device_map={'':device}
    )
    model=model.eval()
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer


def load_qa(dataset_path='/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/NQ-dev.json'):
    if "PopQA" in dataset_path:
        dataset=load_from_disk(dataset_path)
        return dataset['question'],dataset['possible_answers']
    question = []
    answer = []
    with open(dataset_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # 使用json.loads()解析每个JSON对象
                data = json.loads(line)
                # 处理JSON数据
                question.append(data['question'])
                answer.append((data['answer']))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
            except Exception as e:
                print(f"An error occurred: {e}")
    return question, answer

USER_AGENT environment variable not set, consider setting it to identify your requests.
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
embedding_model = load_embedding_model()
knowledge_dir='/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/popQA_knowledge'
knowledge_file_dir='/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/popQA_knowledge.txt'
if not os.path.exists(knowledge_dir):
    documents = load_documents(knowledge_file_dir)
    vector_store = FAISS.from_documents(documents, embedding_model)
    vector_store.save_local(knowledge_dir)
else:
    # 如果本地已经有faiss仓库了，说明之前已经保存过了，就直接读取
    vector_store = FAISS.load_local(knowledge_dir, embeddings=embedding_model, allow_dangerous_deserialization=True)

questions, answers = load_qa("/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/PopQA")
# model, tokenizer = load_Qwen("/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/models/Qwen2.5-7B-Instruct")
model, tokenizer = load_llama("/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/models/Llama-2-7b-chat-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def get_answer_Qwen(question, model, tokenizer, vector_store, device,rag=False):
    if rag:
        docs=vector_store.similarity_search(question, k=6)  # 计算相似度，并把相似度高的chunk放在前面
        contexts = [doc.page_content for doc in docs]
        contexts_text="\n".join(contexts)       
        system_prompt = f"You are now playing the role of the encyclopedia. Answer my questions without any unnecessary words.Here is some knowledge you can refer to:\n{contexts_text}"   
    else:
        system_prompt = "You are now playing the role of the encyclopedia. Answer my questions without any unnecessary words."
    if not question.endswith("?"):
        question = question + "?"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512
        )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    if rag:
        return response,contexts
    else:
        return response
def get_answer_llama2(question, model, tokenizer, vector_store, device, k=6, rag=False):
    if not question.endswith("?"):
        question = question + "?"
    if rag and k>0:
        docs=vector_store.similarity_search(question, k=6)  # 计算相似度，并把相似度高的chunk放在前面
        contexts = [doc.page_content for doc in docs]
        contexts_text="\n".join(contexts)
        input=f"<s>[INST] <<SYS>>\nYou are now playing the role of the encyclopedia. Answer my questions without any unnecessary words.\nYou can use the following knowledge but don't be misled by inaccurate content:\n{contexts_text}\n<</SYS>>nQuestion:\n{question}\nAnswer: [/INST]"       
    else:
        input = f"<s>[INST] <<SYS>>\nYou are now playing the role of the encyclopedia. Answer my questions without any unnecessary words.\n<</SYS>>\nQuestion:\n{question}\nAnswer: [/INST]"
    inputs = tokenizer(input, return_tensors='pt', max_length=4000, truncation=True)
    if inputs["input_ids"].shape[1] > 3900:
        return get_answer_llama2(question, model, tokenizer, vector_store, device, k=k-1, rag=rag)
    with torch.no_grad():
        generate_ids = model.generate(input_ids=inputs["input_ids"].to(device),max_new_tokens=50)
    ans_text=tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    ans_text=ans_text.split("[/INST]")[1].strip()
    if rag:
        if k<=0:
            return ans_text,[]
        return ans_text,contexts
    else:
        return ans_text
        

In [7]:
sample_idx = 30
print(questions[sample_idx])
print(answers[sample_idx])
answer = get_answer_llama2(questions[sample_idx], model, tokenizer, vector_store, device,rag=False)
print("answer:\n",answer)
answer_rag,contexts = get_answer_llama2(questions[sample_idx], model, tokenizer, vector_store, device,rag=True)
print("answer_rag:\n",answer_rag)
print("contexts:\n",contexts)

What is Heath's occupation?
["songwriter", "song writer"]
answer:
 Heath's occupation: Actor
answer_rag:
 Heath is a bassist, composer, singer, and songwriter.
contexts:
 ["Heath: Japanese musician (1968-2023), alias: ['Hiroshi Morie'], instance of: ['human'], image: ['Heath (musician) in São Paulo, Brazil 2011-11-09.jpg532 × 800; 340 KB'], sex or gender: ['male'], country of citizenship: ['Japan'], name in native language: ['HEATH (Japanese)'], birth name: ['森江 博 (Japanese)'], given name: ['Hiroshi'], date of birth: ['22 January 1968'], place of birth: ['Amagasaki'], date of death: ['29 October 2023'], manner of death: ['natural causes'], cause of death: ['colorectal cancer'], languages spoken, written or signed: ['Japanese'], occupation: ['bassist', 'composer', 'singer', 'songwriter'], work period (start): ['1986'], work period (end): ['2023'], blood type: ['B'], instrument: ['bass guitar', 'guitar', 'voice'], member of: ['X Japan'], genre: ['heavy metal'], official website: ['http:/

In [9]:
qa_out_dataset_path = "/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/PopQA_100_llama2_output"
dataset_out = load_from_disk(qa_out_dataset_path)
for i in range(5):
    print(dataset_out['question'][i])
    print(dataset_out['possible_answers'][i])
    print(dataset_out['answer_plain'][i])
    print(dataset_out['answer_rag'][i])
    print("----------------------------")

What is George Rankin's occupation?
["politician", "political leader", "political figure", "polit.", "pol"]
George Rankin's occupation is:

Banker
George Rankin's occupation is politician.
----------------------------
What is John Mayne's occupation?
["journalist", "journo", "journalists"]
John Mayer's occupation is musician.
John Mayne's occupation is journalist, poet, and writer.
----------------------------
What is Henry Feilden's occupation?
["politician", "political leader", "political figure", "polit.", "pol"]
Henry Fielding's occupation is: Author
Henry Feilden's occupation is listed as "Politician".
----------------------------
What is Kathy Saltzman's occupation?
["politician", "political leader", "political figure", "polit.", "pol"]
Kathy Saltzman's occupation is artist.
Kathy Saltzman's occupation is member of the State Senate of Minnesota.
----------------------------
What is Eleanor Davis's occupation?
["cartoonist", "graphic artist", "animator", "illustrator"]
Davis - car

In [10]:

print('ascSCASC'.lower())

ascscasc


In [4]:
sample_idx = 2
print(questions[sample_idx])
print(answers[sample_idx])
answer = get_answer_Qwen(questions[sample_idx], model, tokenizer, vector_store, device,rag=False)
print("answer:\n",answer)
answer_rag,contexts = get_answer_Qwen(questions[sample_idx], model, tokenizer, vector_store, device,rag=True)
print("answer_rag:\n",answer_rag)
print("contexts:\n",contexts)

What is Henry Feilden's occupation?
["politician", "political leader", "political figure", "polit.", "pol"]


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


answer:
 Henry Feilding was an English politician and poet.
answer_rag:
 Henry Feilden's occupation was politician.
contexts:
 ["Henry Feilden: British politician, alias: ['Henry Master Feilden'], instance of: ['human'], sex or gender: ['male'], country of citizenship: ['United Kingdom of Great Britain and Ireland'], name in native language: ['Henry Master Feilden (English)'], given name: ['Henry'], family name: ['Feilden'], date of birth: ['21 February 1818Gregorian'], date of death: ['5 September 1875Gregorian'], father: ['Joseph Feilden'], mother: ['Frances Mary Master'], spouse: ['Caroline Mosley'], child: ['Frances Sophia Feilden'], languages spoken, written or signed: ['English'], occupation: ['politician'], position held: ['member of the 21st Parliament of the United Kingdom', 'member of the 20th Parliament of the United Kingdom'], work location: ['London'], member of political party: ['Conservative Party'], candidacy in election: ['1874 United Kingdom general election'], Freeba

In [1]:
from datasets import load_dataset, load_from_disk
dataset=load_from_disk("/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/PopQA")
dataset=dataset.filter(lambda x,idx:idx<100,with_indices=True)
dataset.save_to_disk("/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/PopQA_100")

Filter:   0%|          | 0/14267 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

6


In [28]:
from datasets import load_dataset,Dataset
import pandas as pd
df = pd.read_csv("/mnt/data/chenjinwen/AI-Security-Evaluation_RAG/data/PopQA/test.tsv", sep='\t')
df = pd.DataFrame(df)
print(df)
dataset = Dataset.from_pandas(df)
print(dataset)
# print(dataset['train'][0])

            id                        subj        prop          obj  subj_id  \
0      4222362               George Rankin  occupation   politician  1850297   
1      4725190                  John Mayne  occupation   journalist  2079053   
2      4382392               Henry Feilden  occupation   politician  1925450   
3      4822110              Kathy Saltzman  occupation   politician  2122743   
4      4011112               Eleanor Davis  occupation   cartoonist  1752619   
...        ...                         ...         ...          ...      ...   
14262  4165948                 Netherlands     capital    Amsterdam  1825504   
14263  2617063  City Municipality of Celje     capital        Celje  1127153   
14264   471864                       Delhi     capital    New Delhi   192624   
14265  2965573     Palestinian territories     capital    Jerusalem  1270077   
14266   802449                   Louisiana     capital  Baton Rouge   342549   

       prop_id   obj_id                

In [15]:
import requests
from bs4 import BeautifulSoup

# 目标URL
url = 'https://www.wikidata.org/wiki/Q2623883'

# 发送GET请求
response = requests.get(url)

# 检查请求是否成功
if response.status_code == 200:
    # 使用BeautifulSoup解析HTML内容
    soup = BeautifulSoup(response.text, 'html.parser')
    # print(soup)
    titles=soup.find_all('div',class_='wikibase-statementgroupview-property')
    tables=soup.find_all('div',class_='wikibase-statementlistview')
    head=soup.find('span',class_='wikibase-title-label')
    head_description=soup.find('div',class_='wikibase-entitytermsview-heading-description')
    aliases=soup.find_all('li',class_='wikibase-entitytermsview-aliases-alias')
    print(head.get_text(),',',head_description.get_text(),',',[alias.get_text() for alias in aliases])
    print(len(tables))
    print(len(titles))
    # print(tables[0])
    for title,table in zip(titles,tables):
        bodys=table.find_all('div',class_='wikibase-snakview-body')
        print(title.get_text().strip(),[x.get_text().strip() for x in bodys])
        print("------------------")
        
    
    # 这里可以添加你想要提取的HTML元素的代码
    # 例如，提取所有的标题
    # items = soup.find_all('wikibase-statementgroupview-property-label')
    # for item in items:
    #     print(item.get_text())
    # items = soup.find_all('wikibase-snakview-body')
    # for item in items:
    #     print(item.get_text())

else:
    print('请求失败，状态码：', response.status_code)
print(response)

William Collins , English landscape and genre painter (1788-1847) , ['William, II Collins', 'William II Collins', 'W. Collins', 'R.A. W. Collins', 'William W. Collins', 'William Collins R. A.', 'Collins', 'R.A. Collins']
90
90
instance of ['human', 'Russian Wikipedia']
------------------
image ['John Linnell - William Collins, R.A. - Google Art Project.jpg4,291 × 5,394; 3.1 MB', 'Commons Creator page']
------------------
sex or gender ['male', 'Swedish Wikipedia', 'Virtual International Authority File', '28432019', '4 November 2018']
------------------
country of citizenship ['United Kingdom of Great Britain and Ireland', 'Kingdom of Great Britain', '1 January 1801Gregorian', 'Acts of Union 1800']
------------------
name in native language ['William Collins (English)']
------------------
given name ['William']
------------------
family name ['Collins']
------------------
date of birth ['8 September 1788Gregorian', 'SNAC', 'w60p13nv', 'William Collins (painter)', '9 October 2017']
-----

In [7]:
import requests
from bs4 import BeautifulSoup
def url2knowledge(url):
    # 发送GET请求
    # print(url)
    headers={
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'
    }
    response = requests.get(url,headers=headers)
    knowledge=''
    # 检查请求是否成功
    if response.status_code == 200:
        # 使用BeautifulSoup解析HTML内容
        # print(response.text)
        soup = BeautifulSoup(response.text, 'html.parser')
        # print(soup)
        titles=soup.find_all('div',class_='wikibase-statementgroupview-property')
        tables=soup.find_all('div',class_='wikibase-statementlistview')
        try:
            assert len(titles)==len(tables)
            # print(url)
        except AssertionError as e:
            print(e)
        head=soup.find('span',class_='wikibase-title-label')
        head_description=soup.find('div',class_='wikibase-entitytermsview-heading-description')
        aliases=soup.find_all('li',class_='wikibase-entitytermsview-aliases-alias')
        if head!=None:
            knowledge+=head.get_text()
            knowledge+=': '
        if head_description!=None:
            knowledge+=head_description.get_text()
            knowledge+=', '
        if aliases!=[]:
            knowledge+="alias: "
            knowledge+=str([alias.get_text() for alias in aliases])
        # print(head.get_text(),',',head_description.get_text(),',',[alias.get_text() for alias in aliases])

        # print(len(tables))
        # print(len(titles))
        # print(tables[0])
        for title,table in zip(titles,tables):
            # bodys=table.find_all('div',class_='wikibase-snakview-body')
            bodys=table.find_all('div',class_='wikibase-statementview-mainsnak-container')
            knowledge+=", "
            knowledge+=title.get_text().strip()
            knowledge+=": "
            knowledge+=str([x.find('div',class_='wikibase-statementview-mainsnak').get_text().strip() for x in bodys])
            # print(title.get_text().strip(),[x.get_text().strip() for x in bodys])
            # print("------------------")
        return knowledge
    else:
        print('请求失败，状态码：', response.status_code)
        return url
# knowledge=url2knowledge("https://www.wikidata.org/wiki/Q2623883")
# with open("./data/popQA_knowledge.txt",'+a') as f:
#     f.write(knowledge)
#     f.write("\n")
# with open()
print(url2knowledge("http://www.wikidata.org/wiki/Q11452861"))


On Practice: philosophical essay by Mao Zedong, , instance of: ['essay'], title: ['实践论 (Chinese)'], genre: ['essay'], author: ['Mao Zedong'], language of work or name: ['Chinese'], start time: ['July 1937'], publication date: ['1937'], Encyclopædia Britannica Online ID: ['topic/On-Practice'], Encyclopedia of China (Third Edition) ID: ['62290'], Freebase ID: ['/m/06p30rx']


In [3]:
from datasets import load_dataset,Dataset,load_from_disk
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import gzip,json
import time
dataset=load_from_disk("D:\BaiduSyncdisk\AI-Security-Evaluation_RAG\data\PopQA")
prograss_bar=tqdm(total=len(dataset['s_uri']))
# print(url2knowledge(str(dataset['s_uri'][0])))
for s_uri in dataset['s_uri']:
    # print(s_uri)
    prograss_bar.update(1)
    knowledge=url2knowledge(s_uri.replace("entity","wiki"))
    # print(knowledge)
    if knowledge==s_uri:
        print(knowledge)
        continue
    with open("D:\BaiduSyncdisk\AI-Security-Evaluation_RAG\data\popQA_knowledge.txt",'+a') as f:
        f.write(knowledge)
        f.write("\n")
    # time.sleep(100)

  0%|          | 1/14267 [00:06<25:08:14,  6.34s/it]


KeyboardInterrupt: 

In [6]:
for i in range(0,11,3):
    print(i)


0
3
6
9
